import pandas as pd

In [1]:
import pandas as pd

In [2]:
tripadv_review = pd.read_csv(r'C:\Users\Dell\Desktop\Restaurant\Louis Tavern & Restaurant.csv', error_bad_lines=False);

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [6]:
processed_reviews = tripadv_review['r_text'].map(preprocess)
processed_reviews

0      [went, louis, tavern, tsiknopempti, dinner, gr...
1      [good, long, time, wide, variety, vegetarian, ...
2      [went, night, time, recommended, couple, times...
3      [went, louis, group, friends, food, nice, good...
4      [husband, visited, restaurant, reading, excell...
                             ...                        
108    [wants, best, quality, meze, town, louis, tave...
109    [restaurant, times, past, knows, standards, lo...
110    [touch, cyprus, best, cypriot, meze, makes, re...
111    [best, meat, meze, tavern, live, paphos, frequ...
112    [cypriot, meze, taverna, great, food, mushroom...
Name: r_text, Length: 113, dtype: object

In [7]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(processed_reviews)

# Create Corpus
texts = processed_reviews

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [8]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=50,
                                           update_every=1,
                                           chunksize=150,
                                           passes=5,
                                           alpha='auto',
                                           eta='auto',
                                           per_word_topics=True,
                                           iterations = 25,
                                           eval_every = None)

In [9]:
from pprint import pprint

In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.021*"food" + 0.020*"meze" + 0.015*"service" + 0.015*"quality" + '
  '0.015*"tavern" + 0.014*"good" + 0.014*"best" + 0.010*"dishes" + '
  '0.010*"excellent" + 0.009*"louis"'),
 (1,
  '0.018*"meze" + 0.010*"cypriot" + 0.009*"dishes" + 0.008*"cooked" + '
  '0.008*"food" + 0.007*"best" + 0.007*"quality" + 0.007*"louis" + '
  '0.006*"souvlaki" + 0.006*"pork"'),
 (2,
  '0.020*"food" + 0.020*"good" + 0.012*"quality" + 0.011*"service" + '
  '0.009*"place" + 0.009*"excellent" + 0.007*"louis" + 0.007*"cyprus" + '
  '0.007*"time" + 0.007*"thank"'),
 (3,
  '0.039*"food" + 0.030*"louis" + 0.017*"great" + 0.014*"meze" + '
  '0.013*"restaurant" + 0.012*"tavern" + 0.011*"nicosia" + 0.010*"friendly" + '
  '0.009*"good" + 0.009*"place"'),
 (4,
  '0.019*"food" + 0.018*"good" + 0.018*"louis" + 0.017*"tavern" + '
  '0.014*"service" + 0.013*"meze" + 0.013*"great" + 0.013*"dishes" + '
  '0.012*"place" + 0.011*"owner"')]


In [11]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.085596  0.032525       1        1  35.861874
0      0.058630 -0.039460       2        1  23.394171
4     -0.028445 -0.024673       3        1  19.905849
1      0.054204  0.082222       4        1  11.632923
2      0.001206 -0.050615       5        1   9.205183, topic_info=          Term       Freq      Total Category  logprob  loglift
14        good  49.000000  49.000000  Default  30.0000  30.0000
175      owner  16.000000  16.000000  Default  29.0000  29.0000
208    quality  36.000000  36.000000  Default  28.0000  28.0000
421    taverns  11.000000  11.000000  Default  27.0000  27.0000
247      local  14.000000  14.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
21        meze   1.561029  60.222847   Topic5  -5.4775  -1.2673
34      tavern   1.509688  49.000298   Topic5  -5.5109  -1.0945
259  delicious   1.361532  14.632279   Topic5  -5.6142   0.0108
39        went   1.368993  18.514000   Topic5  -5.6087  -0.2190
49       visit   1.348127  24.541037   Topic5  -5.6241  -0.5162

[382 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
379       4  0.709969  addition
712       5  0.755711      alot
52        1  0.490923   amazing
52        2  0.070132   amazing
52        3  0.280527   amazing
...     ...       ...       ...
240       3  0.207943     worth
240       5  0.103971     worth
523       1  0.907414     wrong
818       4  0.709596     zesty
717       5  0.755492   zivania

[606 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 5, 2, 3])